In [2]:
import numpy as np
from tqdm import tqdm
import scipy.stats as sp
from scipy.special import gamma
from scipy.optimize import minimize
from scipy.integrate import tplquad 
from jupyprint import jupyprint, arraytex
from numdifftools import Hessian, Jacobian
from IPython.display import clear_output
import matplotlib.pyplot as plt

In [3]:
# Poisson Random Effects model 

# parameters of the model 
sigma_mu = 2 # (standard deviation)
I = 5
J = 6

In [4]:
# sample observations:
mu = 1
eta = np.random.normal(loc=mu, scale=1, size=I)
y = np.zeros((I,J))

for i in range(I):
    y[i,:] = np.random.poisson(lam=np.exp(eta[i]), size=J)


In [5]:
# define potential and gradient and hessian 
def potential(x):
    output = J*np.sum(np.exp(x[1:])) + np.sum(x[1:].dot(y)) + 1/2 * np.sum((x[1:]-x[0])**2) + x[0]**2 / (2* sigma_mu**2)
    return output

def gradient(x):
    output = np.zeros(len(x))
    output[0] = np.sum(x[0] - x[1:]) + x[0]/sigma_mu**2
    output[1:] = J * np.exp(x[1:]) + x[1:] - x[0]  + np.sum(y,axis=1)
    return output

def hessian_p(x,p):
    output = np.zeros(len(x))
    output[0] = (I+1/sigma_mu**2)*p[0] - np.sum(p[1:])
    output[1:] = -p[0] +(J * np.exp(x[1:]) + 1)*p[1:]
    return output


In [8]:
# test potential and gradient
x0 = sp.norm.rvs(size=d)
jupyprint("$f(x_0) =" + str(potential(x0)) +"$")
#jupyprint("$f^\\prime(x_0) =" + str(gradient(x0)) +"$")
#jupyprint("$\\hat{f}^\\prime(x_0) =" + str(Jacobian(potential)(x0)) +"$")

def density_c(x):
    return np.exp(-potential(x))


$f(x_0) =261.01495602146485$

In [22]:
# PARAMETERS OF THE RUN
d = I+1
initial_sample = np.zeros(d)
time_step = 1e-4
sampling = 10000
n_MC = 1
tune_interval = 10000
number_of_samples = sampling+tune_interval

In [13]:
# Metropolis algorithm

# Monte Carlo 
moment2_metropolis = []
moment4_metropolis = []
moment6_metropolis = []

test_rejections = 0

for i_MC in range(0, n_MC):
    samples_metropolis = [initial_sample]
    samples_metropolis_norm = [np.linalg.norm(initial_sample)]

    for _ in tqdm(range(1, number_of_samples)):

        # get proposal
        proposal = samples_metropolis[-1] + sp.norm.rvs(
            loc=0, 
            scale=0.005, 
            size=d
            )
        
        if density_c(samples_metropolis[-1]) == 0:
            alpha = 1
        else:
            alpha = density_c(proposal)/density_c(samples_metropolis[-1])

        u = sp.uniform.rvs()

        if u <= alpha:
            samples_metropolis.append(proposal)
            samples_metropolis_norm.append(np.linalg.norm(proposal))
        else:
            samples_metropolis.append(samples_metropolis[-1])
            samples_metropolis_norm.append(samples_metropolis_norm[-1])
            test_rejections += 1

    print("acceptance rate = "+str(1-test_rejections/number_of_samples))
    test_rejections = 0

    samples_metropolis_norm = np.array(samples_metropolis_norm[tune_interval:])

    moment2_metropolis.append(np.mean(samples_metropolis_norm**2))
    moment4_metropolis.append(np.mean(samples_metropolis_norm**4))
    moment6_metropolis.append(np.mean(samples_metropolis_norm**6))

    jupyprint("Metropolis")
    jupyprint("2nd: mean = " + str(np.mean(moment2_metropolis)) + ", std = " + str(np.std(moment2_metropolis)))
    jupyprint("4th: mean = " + str(np.mean(moment4_metropolis)) + ", std = " + str(np.std(moment4_metropolis)))
    jupyprint("6th: mean = " + str(np.mean(moment6_metropolis)) + ", std = " + str(np.std(moment6_metropolis)))

  8%|▊         | 9079/109999 [00:02<00:22, 4516.26it/s]C:\Users\Benko\AppData\Local\Temp\ipykernel_26808\2341137570.py:8: RuntimeWarning: overflow encountered in exp
  return np.exp(-potential(x))
C:\Users\Benko\AppData\Local\Temp\ipykernel_26808\1041804514.py:26: RuntimeWarning: invalid value encountered in scalar divide
  alpha = density_c(proposal)/density_c(samples_metropolis[-1])
100%|██████████| 109999/109999 [00:24<00:00, 4497.80it/s]

acceptance rate = 0.07089999999999996


Metropolis

2nd: mean = 66.12162025032309, std = 0.0

4th: mean = 4372.068664527943, std = 0.0

6th: mean = 289088.2639442542, std = 0.0

In [31]:
# EXPLICIT TAMED SCHEME

# Monte Carlo 
moment2_explicit_tamed = []
moment4_explicit_tamed = []
moment6_explicit_tamed = []

for i_MC in range(0, n_MC):
    print("MC: "+str(i_MC+1)+"/"+str(n_MC))

    samples_explicit_tamed = [initial_sample]
    samples_explicit_tamed_norm = [np.linalg.norm(initial_sample)]

    for _ in tqdm(range(1, number_of_samples)):

        # gradient tamed step
        x = samples_explicit_tamed[-1] - time_step * gradient(samples_explicit_tamed[-1]) / (1+time_step*np.linalg.norm(gradient(samples_explicit_tamed[-1]))) 
        
        # adding Gaussian
        cov = np.diag(np.repeat(2*time_step,d))
        x = sp.multivariate_normal.rvs(mean=x, cov=cov)
        # save value
        samples_explicit_tamed.append(x)
        samples_explicit_tamed_norm.append(np.linalg.norm(x))

    samples_explicit_tamed_norm = np.array(samples_explicit_tamed_norm[tune_interval:])

    moment2_explicit_tamed.append(np.mean(samples_explicit_tamed_norm**2))
    moment4_explicit_tamed.append(np.mean(samples_explicit_tamed_norm**4))
    moment6_explicit_tamed.append(np.mean(samples_explicit_tamed_norm**6))

MC: 1/1


100%|██████████| 19999/19999 [00:09<00:00, 2048.33it/s]


In [46]:
# EXPLICIT SCHEME 

# Monte Carlo 
moment2_explicit = []
moment4_explicit = []
moment6_explicit = []

for i_MC in range(0, n_MC):
    print("MC: "+str(i_MC+1)+"/"+str(n_MC))

    samples_explicit = [initial_sample]
    samples_explicit_norm = [np.linalg.norm(initial_sample)]

    for _ in tqdm(range(1, number_of_samples)):

        #gradient step
        x = samples_explicit[-1] - time_step * gradient(samples_explicit[-1])  
        
        # adding Gaussian
        cov = np.diag(np.repeat(2*time_step,d))
        x = sp.multivariate_normal.rvs(mean=x, cov=cov)
        # save value
        samples_explicit.append(x)
        samples_explicit_norm.append(np.linalg.norm(x))

    samples_explicit_norm = np.array(samples_explicit_norm[tune_interval:])

    moment2_explicit.append(np.mean(samples_explicit_norm**2))
    moment4_explicit.append(np.mean(samples_explicit_norm**4))
    moment6_explicit.append(np.mean(samples_explicit_norm**6))

MC: 1/10


100%|██████████| 109999/109999 [00:29<00:00, 3765.77it/s]


MC: 2/10


100%|██████████| 109999/109999 [00:30<00:00, 3637.62it/s]


MC: 3/10


...

100%|██████████| 109999/109999 [00:30<00:00, 3570.84it/s]


MC: 9/10


100%|██████████| 109999/109999 [00:29<00:00, 3685.24it/s]


MC: 10/10


100%|██████████| 109999/109999 [00:30<00:00, 3612.08it/s]


In [23]:
# IMPLICIT INEXACT SCHEME 

# Monte Carlo 
moment2_implicit = []
moment4_implicit = []
moment6_implicit = []

for i_MC in range(0, n_MC):
    clear_output(wait=True)
    print("MC: "+str(i_MC+1)+"/"+str(n_MC))

    samples_implicit = [initial_sample]
    samples_implicit_norm = [np.linalg.norm(initial_sample)]

    for _ in tqdm(range(1, number_of_samples)):

        # inexact proximal step 
        x = minimize(
            lambda x: potential(x) + 1/(2*time_step) * np.linalg.norm(x - samples_implicit[-1])**2, 
            jac=lambda x: gradient(x) + 1/time_step * (x - samples_implicit[-1]),
            x0=samples_implicit[-1]
            ).x
        
        # adding Gaussian
        cov = np.diag(np.repeat(2*time_step,d))
        x = sp.multivariate_normal.rvs(mean=x, cov=cov)
        # save value
        samples_implicit.append(x)
        samples_implicit_norm.append(np.linalg.norm(x))

    samples_implicit_norm = np.array(samples_implicit_norm[tune_interval:])

    moment2_implicit.append(np.mean(samples_implicit_norm**2))
    moment4_implicit.append(np.mean(samples_implicit_norm**4))
    moment6_implicit.append(np.mean(samples_implicit_norm**6))

jupyprint("results")
jupyprint("implicit")
jupyprint("2nd: mean = " + str(np.mean(moment2_implicit)) + ", std = " + str(np.std(moment2_implicit)))
jupyprint("4th: mean = " + str(np.mean(moment4_implicit)) + ", std = " + str(np.std(moment4_implicit)))
jupyprint("6th: mean = " + str(np.mean(moment6_implicit)) + ", std = " + str(np.std(moment6_implicit)))

MC: 1/1


100%|██████████| 19999/19999 [00:42<00:00, 469.95it/s]


results

implicit

2nd: mean = 11247.920604826095, std = 0.0

4th: mean = 139603582.77516592, std = 0.0

6th: mean = 1872257285819.172, std = 0.0

In [29]:
# PRINT RESULTS 
# print reference 

# jupyprint("explicit")
# jupyprint("2nd: mean = " + str(np.mean(moment2_explicit)) + ", std = " + str(np.std(moment2_explicit)))
# jupyprint("4th: mean = " + str(np.mean(moment4_explicit)) + ", std = " + str(np.std(moment4_explicit)))
# jupyprint("6th: mean = " + str(np.mean(moment6_explicit)) + ", std = " + str(np.std(moment6_explicit)))
jupyprint("explicit tamed")
jupyprint("2nd: mean = " + str(np.mean(moment2_explicit_tamed)) + ", std = " + str(np.std(moment2_explicit_tamed)))
jupyprint("4th: mean = " + str(np.mean(moment4_explicit_tamed)) + ", std = " + str(np.std(moment4_explicit_tamed)))
jupyprint("6th: mean = " + str(np.mean(moment6_explicit_tamed)) + ", std = " + str(np.std(moment6_explicit_tamed)))


results

implicit

2nd: mean = 1844232.1098224078, std = 0.0

4th: mean = 3401405217647.229, std = 0.0

6th: mean = 6.273765719968522e+18, std = 0.0

explicit tamed

2nd: mean = 1840908.7704516256, std = 0.0

4th: mean = 3389285618962.8945, std = 0.0

6th: mean = 6.240605494770248e+18, std = 0.0

In [26]:
# plot trajectory of some coordinate 
plt.figure()

coord_draw = 2

# plt.style.use(['science'])
# plt.rcParams.update({
#   #  "font.family": "serif",   # specify font family here
#   #  "font.serif": ["Times"],  # specify font here
#     "text.usetex": True,
#     } 
#     )

# # plt.rcParams.update({
# #   "text.usetex": True,
# #   "font.family": "serif", 
# #   "font.size":12
# # })

# plot trajectory of some coordinate 
sel1 = 0
sel2 = 20000

samples_metropolis = np.array(samples_metropolis)
plt.figure()
plt.plot(np.arange(sel1, sel2), samples_metropolis[sel1:sel2, coord_draw])
plt.title("Metropolis")

# plt.figure(figsize=(3,2))
# plt.plot(np.arange(sel1, sel2), samples_explicit[sel1:sel2, coord_draw])
# plt.xlabel("$k$")
# plt.ylabel("$x_{"+str(coord_draw)+"}$")
# #plt.xticks(np.array([0,1,2,3]))
# #plt.title("explicit")
# #plt.savefig("explicit2.pdf")

# plt.figure(figsize=(3,2))
# plt.plot(np.arange(sel1, sel2), samples_explicit_tamed[sel1:sel2, coord_draw])
# plt.xlabel("$k$")
# plt.ylabel("$x_{"+str(coord_draw)+"}$")
# #plt.title("explicit tamed")
# plt.savefig("explicit_tamed.pdf")

samples_implicit = np.array(samples_implicit)
plt.figure(figsize=(3,2))
plt.plot(np.arange(sel1, sel2), samples_implicit[sel1:sel2, coord_draw])
plt.xlabel("$k$")
plt.ylabel("$x_{"+str(coord_draw)+"}$")
# #plt.title("implicit")
# plt.savefig("implicit.pdf")



TypeError: list indices must be integers or slices, not tuple

<Figure size 640x480 with 0 Axes>

<Figure size 640x480 with 0 Axes>